# 1.Setup Environment for OpenAI

Store `OPENAI_API_KEY` as an environment variable in a `.env` file and load it using [pytyon-dotenv](https://pypi.org/project/python-dotenv/)

In [9]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
gpt_model_name = "gpt-3.5-turbo-1106"
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
)


In [10]:
def get_transcript(file_path):
    audio_file= open(file_path, "rb")
    transcript = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="text"
    )
    return transcript

In [11]:
system_prompt = """
You are an experienced English tutor who graduated from Harvard University in Boston.
You are talking to a student who wants to practice speaking English. 
Help them practice speaking English by talking to your student and 
try to teach your student how to say what they would like to say.
The answer must be formatted as a JSON string
"""

def get_gpt_response(transcript, history):
  
  system_message = {
    "role": "system", 
    "content": system_prompt.replace("\n", " ")
  }
  
  message_list = [system_message]
  message_list.extend(history)
  message_list.append({"role": "user", "content": transcript})

  response = client.chat.completions.create(
    model=gpt_model_name,
    response_format={ "type": "json_object" },
    messages=message_list
  )
  
  return response.choices[0].message.content


In [12]:
import os
from playsound import playsound


def play_gpt_response_with_tts(gpt_response):
    speech_file_path = "./speech.mp3"
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=gpt_response
    )

    response.stream_to_file(speech_file_path)
    playsound(speech_file_path)
    os.remove(speech_file_path)
    

In [13]:
import json

history = []


def talk_to_gpt(file_path):
    user_transcript = get_transcript(file_path)
    gpt_response = get_gpt_response(user_transcript, history[-10:])
    gpt_response = json.loads(gpt_response)
    gpt_response = gpt_response['response']
    history.extend([
        {"role": "user", "content": user_transcript}, 
        {"role": "assistant", "content": gpt_response}
    ])
    print(gpt_response)
    play_gpt_response_with_tts(gpt_response=gpt_response)

In [14]:
import threading
import sounddevice as sd
import numpy as np
import wavio
import ipywidgets as widgets
from IPython.display import display


In [15]:
class AudioRecorder:
    def __init__(self):
        self.is_recording = False
        self.audio_data = []
        self.fs = 44100  # Sample rate
        self.channels = 1  # Channels

    def start_recording(self):
        self.is_recording = True
        self.audio_data = []
        threading.Thread(target=self.record).start()

    def stop_recording(self):
        self.is_recording = False

    def record(self):
        with sd.InputStream(samplerate=self.fs, channels=self.channels) as stream:
            while self.is_recording:
                data, _ = stream.read(1024)
                self.audio_data.append(data)

    def save(self, filename='output.wav'):
        if self.audio_data:
            wav_data = np.concatenate(self.audio_data, axis=0)
            wavio.write(filename, wav_data, self.fs, sampwidth=2)
            print("Recording saved to", filename)
            return filename
        else:
            print("No recording data to save.")


In [16]:
recorder = AudioRecorder()

start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")

def on_start_clicked(b):
    recorder.start_recording()
    print("Recording started...")

def on_stop_clicked(b):
    recorder.stop_recording()
    file_name = recorder.save()
    talk_to_gpt(file_name)
    print("Recording stopped and saved.")

start_button.on_click(on_start_clicked)
stop_button.on_click(on_stop_clicked)

display(start_button, stop_button)


Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Recording started...
Recording saved to output.wav
Hi Jason, it's great to meet you! I can definitely help you with your English. What specifically would you like to work on today?
Recording stopped and saved.
Recording started...
Recording saved to output.wav
Great, small talk is an important skill. Let's start with some basic conversation starters. You can say things like, 'How was your weekend?' or 'Did you do anything fun last night?' These are nice ways to start a conversation with your colleagues. Let's practice saying these phrases together.
Recording stopped and saved.
Recording started...


/Users/jasonkang/.pyenv/versions/3.10.12/envs/openai-whisper/lib/python3.10/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))


Recording saved to output.wav
Hey, so how was your weekend?
Recording stopped and saved.
Recording started...
Recording saved to output.wav
Oh, my weekend was great, thanks for asking! I went for a hike and spent some time with my family. How about you? Did you do anything fun?
Recording stopped and saved.
